# CS5830 Project 8: Decision Trees/Neural Networks

In [ ]:
from pathlib import Path
output_dir = Path('output')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix 
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from IPython.display import SVG
from graphviz import Source
import networkx as nx
import colorsys

In [ ]:
df = pd.read_csv("diabetes.csv")
df.info()

In [ ]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]
X

## Explore feature correlations

In [ ]:
corr = X.corr()
plt.figure(figsize=(12, 10))
heatmap = sns.heatmap(
    corr,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    annot_kws={"fontsize": 16}
)  # Adjust fontsize as needed

heatmap.set_xticklabels(heatmap.get_xticklabels(),  fontsize=16)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=16)
plt.title("Correlation Heatmap", fontsize=19)  # Adjust fontsize as needed
plt.show()

In [ ]:
X = X[["Glucose", "BMI", "Age", "Pregnancies", "DiabetesPedigreeFunction"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
display(X_train)

## Decision Tree: `max_depth=3`

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=1)
dt.fit(X_train, y_train)

p, r, f, _ = precision_recall_fscore_support(y_test, dt.predict(X_test), labels=[0, 1])
display(pd.DataFrame({
    "Precision": p,
    "Recall": r,
    "F1": f
    }, index=["Class 0", "Class 1"]).T)

dot = export_graphviz(
    dt,
    out_file=None,
    feature_names=["Glucose", "BMI", "Age", "Pregnancies", "DiabetesPedigreeFunction"],
    class_names=["0", "1"],
    filled = True
)

graph = Source(dot)
svg = SVG(graph.pipe(format='svg'))
# plt.savefig("tree3.jpg")
display(svg)
svg_bytes = graph.pipe(format='png')

filename = "decision_tree3.png"
with open(output_dir / filename, "wb") as f:
    f.write(svg_bytes)

## Decision Tree: `max_depth=9`

In [ ]:
dt = DecisionTreeClassifier(max_depth=9, random_state=1)
dt.fit(X_train, y_train)

p, r, f, _ = precision_recall_fscore_support(y_test, dt.predict(X_test), labels=[0, 1])
display(pd.DataFrame({
    "Precision": p,
    "Recall": r,
    "F1": f
    }, index=["Class 0", "Class 1"]).T)

dot = export_graphviz(
    dt,
    out_file=None,
    feature_names=["Glucose", "BMI", "Age", "Pregnancies", "DiabetesPedigreeFunction"],
    class_names=["0", "1"],
    filled = True
)

graph = Source(dot)
svg = SVG(graph.pipe(format='svg'))
display(svg)
svg_bytes = graph.pipe(format='png')

# Write the SVG bytes to a file
filename = "decision_tree5.png"
with open(output_dir / filename, "wb") as f:
    f.write(svg_bytes)

## Neural Networks

In [ ]:
# Function defs

def mlp_info(mlp: MLPClassifier):
    print('This dataset has {} input nodes and {} output node(s)'.format(len(X.columns), len(y.unique())))
    print('There are {} 2D arrays of coefficients, one for each layer'.format(len(mlp.coefs_)))
    print('The layers have the following number of coefficients: {}')
    for l in range(len(mlp.coefs_)):
        m = len(mlp.coefs_[l])
        n = len(mlp.coefs_[l][0])
        print('  {}: {}x{} ({} nodes feeding into a layer of {} nodes)'.format(l, m, n, m, n))
    # Print the actual coefficients
    # print(mlp.coefs_)

    print()
    print('There are {} 1D arrays of intercepts, one for each layer'.format(len(mlp.intercepts_)))
    print('Each layer has {} intercepts, one for each node'.format([len(mlp.intercepts_[l]) for l,_ in enumerate(mlp.intercepts_)]))


def show_ann(mlp):
    hidden_layers_n = len(mlp.coefs_)-1
    layers_n = hidden_layers_n + 2
    input_neurons_n = len(mlp.coefs_[0])
    hidden_neurons_n = [len(mlp.coefs_[i+1]) for i in range(hidden_layers_n)]
    output_neurons_n = len(mlp.coefs_[-1][0])

    G = nx.DiGraph()
    pos = {}

    # Create the neurons of the input layer
    for i in range(input_neurons_n):
        pos['Layer0_{}'.format(i)] = (i,layers_n-1)

    for j in range(hidden_layers_n):
        # Create the neurons of the j'th hidden layer
        prev_layer = j
        cur_layer = j+1
        if (j == 0):
            prev_size = input_neurons_n
        else:
            prev_size = hidden_neurons_n[j-1]
        for i in range(hidden_neurons_n[j]):
            pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
            for k in range(prev_size):
                w = mlp.coefs_[prev_layer][k][i]
                G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    # Create the neurons of the output layer
    prev_layer = hidden_layers_n
    cur_layer = hidden_layers_n+1
    for i in range(output_neurons_n):
        pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
        for k in range(hidden_neurons_n[-1]):
            w = mlp.coefs_[prev_layer][k][i]
            G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    edges = G.edges()
    colors = [colorsys.hsv_to_rgb(0 if G[u][v]['weight'] < 0 else 0.65,
                                  1,#min(1, abs(G[u][v]['weight'])),
                                  1) for u,v in edges]
    weights = [abs(G[u][v]['weight'])*2 for u,v in edges]

    nx.draw(
        G,
        pos,
        node_color="y",
        node_size=150,
        width=weights,
        edge_color=colors
    )
    

### Scale the feature data

In [ ]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler().set_output(transform="pandas")
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

### Train and fit

#### `hidden_layer_sizes=(10, 14, 6)`

In [ ]:
mlp = MLPClassifier(
    max_iter=1000,
    hidden_layer_sizes=(10, 14, 6),
    random_state=1,
)

mlp.fit(X_train,y_train)

#### Prediction/Evaluation

In [ ]:
y_pred = mlp.predict(X_test)

print(classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d")
plt.title("Confusion Matrix for Neural Network")
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
mlp_info(mlp)
show_ann(mlp)

#### `hidden_layer_sizes=(5, 8, 11, 15)`

In [ ]:
mlp = MLPClassifier(
    max_iter=1000,
    hidden_layer_sizes=(5, 8, 11, 15),
    random_state=1,
)

mlp.fit(X_train,y_train)

#### Prediction/Evaluation

In [ ]:
y_pred = mlp.predict(X_test)

print(classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d")
plt.title("Confusion Matrix for Neural Network")
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
mlp_info(mlp)
show_ann(mlp)

#### `hidden_layer_sizes=(5, 8)`

In [ ]:
mlp = MLPClassifier(
    max_iter=1000,
    hidden_layer_sizes=(5, 8),
    random_state=1,
)

mlp.fit(X_train,y_train)

#### Prediction/Evaluation

In [ ]:
y_pred = mlp.predict(X_test)

print(classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d")
plt.title("Confusion Matrix for Neural Network")
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
mlp_info(mlp)
show_ann(mlp)